# Auto Differentiation Package

- **Pytorch supports reverse-mode auto-differentiation**

- Dynamic execution / graph construction **(CG)** (Unlike TF or Caffe)

- No need for forward graph

- Allows in-place operations

- Disjoint graphs. Only include relevant subset of the CG

- - -

### Eager execution

- Red squares: Tensors

- Gray circles: Operations

- - -

- **Forward**
<img src="CG.png" width="400">

- **Backward**

<img src="CG.jpg" width="400">


In [ ]:
import torch

B = 10

# Allow grad flow
x = torch.randn(B, 10, requires_grad=True)
# No grad flow to y
y = torch.randn(B, 5)

def f(x, y):
    z = x @ y    # Matmul          : (10x10),(10x5) -> (10x5)
    t = z ** 2   # Power           : (10x5) -> (10x5)
    z = t + z    # Elementwise sum : (10x5),(10x5) -> (10x5)

    z = z.sum(1) # Sum at axis 1   : (10x5) -> (10)
    return z

## Gradient of a scalar output or ...

Pytorch allows Jacobian vector product but it doesn't calculate Jacobian of $f$.

If the output dimension is just the batch dimension, where we need to accumulate, this is not a problem. This way of auto-differentiation(AD) is very efficient and sufficient(most of the time) in **DL**.

$$ f(x, y): R^{10x10}xR^{10x5} \rightarrow R^{10} $$

#### Jacobian Vector Product (jvp)

In [ ]:
# Vector in jvp
vectr_in_jvp = torch.ones(B)

# Jacobian Vector Product calculation
f(x, y).backward(vectr_in_jvp)

# Storing the gradient
jacob_vec_prod_grad = x.grad.clone()
x.grad.zero_()

# Output size of function $f$
f(x, y).shape

#### Gradient(differentiation in scalar output functions) 

In [ ]:
# f(.).sum() := scalar output function
scalar_out = f(x, y).sum()
scalar_out.backward()

# Storing the gradient
scalar_out_grad = x.grad.clone()

#### Comparing gradients

In [ ]:
(jacob_vec_prod_grad == scalar_out_grad).all().item()

## Neural Networks

- torch.nn
- torch.optim

Pytorch has a rich function and tool set for Neural networks.

The core component is ```torch.nn.Module```.

Please check: https://github.com/pytorch/pytorch/blob/master/torch/nn/modules/module.py

In [ ]:
class Net(torch.nn.Module):
    """ Simple fully connected neural network with 
    Layer normalization layers.
    Arguments:
        - in_size: Input size
        - out_size: Output size
    """
    
    def __init__(self, in_size, out_size):
        super().__init__()
        
        self.lin1 = torch.nn.Linear(in_size, 64)
        self.nrm1 = torch.nn.LayerNorm(64)
        self.rel1 = torch.nn.ReLU()
        self.lin2 = torch.nn.Linear(64, 64)
        self.nrm2 = torch.nn.LayerNorm(64)
        self.rel2 = torch.nn.ReLU()
        self.lin3 = torch.nn.Linear(64, out_size)
        
        self.layers = (self.lin1, self.nrm1, self.rel1, self.lin2, self.nrm2, self.rel2, self.lin3)
        
        self.in_size = in_size
        self.out_size = out_size
        
#         self.param_init()
        
    def forward(self, x):
        """ Forward function of the network.
        """
        for layer in self.layers:
            x = layer(x)
        return x
    
    @staticmethod
    def param_init(module):
        """ Parameter initialization with relu gain.
        """
        # Obtain gain for xavier initializer
        gain = torch.nn.init.calculate_gain("relu")
        if isinstance(module, torch.nn.Linear):
            torch.nn.init.xavier_uniform_(module.weight, gain)
            if module.bias is not None:
                torch.nn.init.zeros_(module.bias)

#### Initialization

When the super class is ```torch.nn.Module```, any attribute initiated within the class is registered according to:

```
def __setattr__(self, name, value):
    ...
```

This method simply checks if:

- Attribute is a ```Module``` object.

    If so, it is registered to the ```_modules``` dict.

- Attribute is a ```Parameter``` object

    It is registered into ```_parameters``` dict.

- - -

This is how pytorch knows all the modules and parameters within the class(recursively)

In [ ]:
net = Net(4, 2)

# All the modules registered during initialization
net._modules

```apply``` function is used to apply a function, that takes one input(module), recursively.

In [ ]:
std_before = net.lin1.weight.std()
# Apply initialization to all modules(recursively)
net.apply(net.param_init)
std_after = net.lin1.weight.std()

# Change the device of all the modules(recursively) in the "net" object
net.apply(lambda m: m.cuda())
net.apply(lambda m: m.cpu())
std_before, std_after

#### Forward

When the ```Module``` class instance(net in our example) is called, python calls ```__call__()``` method. Pytorch ```Module``` objects call the ```forward``` method in the ```__call__``` method which is overridden by the super class(Module). But it also checks and calls followings:

- If there is a hook registered, the callback function is called with necessary arguments

    - backward_hook: Called at the backward pass
    - forward_hook: Called at the end of the forward pass
    - forward_pre_hook: Called at the beginning of the forward pass
    
- Call and return ```forward```

**Note**: Hooks are cool and useful but we will not mention them in this tutorial.

In [ ]:
input = torch.rand(B, 4)
output = net(input)           # __call__ is called with all the checks and hook functionality. (safer)
output = net.forward(input)   # only forward is called
output, output.shape

## DQN

Vanilla DQN implementation. The loss we will be minimizing is semi-gradient loss given below.

$$
    TD = || \max_{a_i} Q(s', a_i: \theta') + r - Q(s, a: \theta)  ||_{2}^{2}
$$

It is called **semi**-gradient since we do not take the derivative w.r.t target network as well. Instead, we use another network(seperate parameters $\theta'$) to find the target values.

In [ ]:
import torch
import random
import numpy as np
from copy import deepcopy
from collections import namedtuple
import gym

In [ ]:
print({
    "torch": torch.__version__,
    "numpy": np.__version__,
    "gym": gym.__version__
})

We need a replay buffer of transitions

In [ ]:
from random import sample as randsample

Transition = namedtuple("Transition", ("state",
                                        "action",
                                        "reward",
                                        "next_state",
                                        "terminal")
                                    )

class UniformBuffer(object):
    """ Queue implementation with lists that allows random
    sampling.
    """

    def __init__(self, size):
        self.queue = []
        self.cycle = 0
        self.size = size

    def __len__(self):
        return len(self.queue)

    def push(self, **transition):
        """ Push the transition keyword arguments into queue.
        If list is full, then overwrite the oldest transition. Else,
        append the transition.
        """
        if self.size != len(self.queue):
            self.queue.append(Transition(**transition))
        else:
            self.queue.append(Transition(**transition))
            self.cycle  = (self.cycle + 1)%self.size
    
    def sample(self, batchsize):
        """ Take a sample from the queue. If batchsize is larger than
        the queue itself than return None.
        """
        if batchsize > len(self.queue):
            return None
        batch = randsample(self.queue, batchsize)
        return Transition(*zip(*batch))

In [ ]:
class Dqn(torch.nn.Module):
    """ Dqn implementation.
    Arguments:
        - valuenet: Neural network(torch.nn.Module) that will be used
        for value estimation.
        - buffersize: Size of the replay buffer.
    """
    def __init__(self, valuenet, buffersize=10000):
        super().__init__()
        self.buffer = UniformBuffer(buffersize)
        self.network = valuenet
        self.targetnet = deepcopy(valuenet)
        self.device = "cpu"
        self.update_count = 0

    def act(self, state, epsilon=0.0):
        """ Epsilon delta policy.
        Arguments:
            - state: Batch size 1 torch tensor.
            - epsilon: Random action probability.
        """
        with torch.no_grad():
            values = self.network(state)
        values = values.squeeze()
        if len(values.shape) > 1:
            raise ValueError("Batch size of the given stat tensor for act must be 1!")
        if random.uniform(0, 1) < epsilon:
            action = torch.randint(values.shape[-1], (1,)).long()
            value = values[action]
        else:
            action = torch.argmax(values)
            value = values[action]
        return (action.item(), value.item())
        
    def td_loss(self, gamma, batchsize):
        """ TD loss of the semi-gradient Q learning.
        Take a sample from the replay memory of size "batchsize".
        Arguments:
            - gamma: Discount factor
            - batchsize: Batch size
        """
        # Take a sample of transitions(iid assumption holds)
        batch = self.buffer.sample(batchsize)
        batch = self._batchtotorch(batch)

        # Values of the next state are taken so that the gradient flow is 
        # disabled in"next_values". Because we use Semi-gradient q learning
        # (a.k.a. Q Learning in function approximation).
        with torch.no_grad():
            next_values = self.targetnet(batch.next_state)
            next_values = torch.max(next_values, dim=1, keepdim=True)[0]

        current_values = self.network(batch.state)
        current_values = current_values.gather(1, batch.action)

        target_value = next_values*(1 - batch.terminal)*gamma + batch.reward
        # We could use l2 loss or l1 loss as well. Remember that, the choice is 
        # environment and algorithm dependent.
        td_error = torch.nn.functional.smooth_l1_loss(current_values, target_value)

        return td_error

    def update(self, opt, gamma, batchsize, target_update_period, grad_clip=False):
        """ Update one step with the optimizer.
        """

        # Update target network at every "target_update_period"
        if self.update_count >= target_update_period:
            self.update_count = 0
            self.targetnet.load_state_dict(self.network.state_dict())
        
        opt.zero_grad()
        loss = self.td_loss(gamma, batchsize)
        loss.backward()

        # Clip the gradients to prevent gradients to diverge
        if grad_clip:
            for param in self.network.parameters():
                param.grad.data.clamp_(-1, 1)
            
        opt.step()
        self.update_count += 1
        return loss.item()

    def push(self, state, action, reward, next_state, terminal):
        self.buffer.push(**dict(state=state,
                                action=action,
                                reward=reward,
                                next_state=next_state,
                                terminal=terminal))

    def _batchtotorch(self, batch):
        """ Helper method to convert raw batch into tensors.
        """
        state = self._totorch(batch.state, torch.float32)
        action = self._totorch(batch.action, torch.long).view(-1, 1)
        next_state = self._totorch(batch.next_state, torch.float32)
        terminal = self._totorch(batch.terminal, torch.float32).view(-1, 1)
        reward = self._totorch(batch.reward, torch.float32).view(-1, 1)
        return Transition(state, action, reward, next_state, terminal)

    def _totorch(self, container, dtype):
        tensor = torch.tensor(container, dtype=dtype)
        return tensor.to(self.device)

    def to(self, device):
        super().to(device)
        self.device = device

Train a DQN agent in LunarLander

In [ ]:
def train(args, net, opt, env):
    # Convert the model into given "device"
    agent.to(args.device)
    
    # List of the rewards
    reward_list = []

    for eps in range(args.episodes):
        state = env.reset()
        
        # Decay the epsilon so that the policy can converge
        eps_reward = 0
        episode_value = 0
        epsilon = (args.start_epsilon-args.end_epsilon)*(1-eps/args.episodes) + args.end_epsilon
        
        for i in range(args.maxlength):
            torch_state = agent._totorch(state, torch.float32).view(1, -1)
            action, value = agent.act(torch_state, epsilon)
            next_state, reward, done, _ = env.step(action)
            agent.push(state, action, reward, next_state, done)
            
            # Gather statistics
            eps_reward += reward
            episode_value += value

            # Start updating when the replay memory has enough samples
            if len(agent.buffer) > args.batchsize:
                td_loss = agent.update(opt, args.gamma, args.batchsize, args.target_period, args.clipgrad)

            if done:
                break
            state = next_state
        
        # End of a trajectory
        reward_list.append(eps_reward)
        print("Episode: {}, avg epsiodic reward: {}".format(eps, np.mean(reward_list[-10:])))
    return reward_list

#### Hyperparameters

In [ ]:
# You can play with the hyperparameters
hyperparams = {
    "lr": 0.001,
    "gamma": 0.98,
    "episodes": 300,
    "buffersize": 10000,
    "target_period": 50,
    "batchsize": 128,
    "start_epsilon": 0.9,
    "end_epsilon": 0.1,
    "maxlength": 300,
    "clipgrad": True,
    "device": "cpu",
    "seed": 12020,    # Holocene year(current year + 10000)
}


# Use namedtuple instead of a dictionary(ofcourse optional)
HyperParams = namedtuple("HyperParams", hyperparams.keys())
args = HyperParams(**hyperparams)

# Initiate enviroment
env = gym.make("LunarLander-v2")
insize = env.observation_space.shape[0]
outsize = env.action_space.n

# Seed
torch.manual_seed(args.seed)
random.seed(args.seed)
np.random.seed(args.seed)
env.seed(args.seed)

# Initiate agent and optimizer
net = Net(insize, outsize)
agent = Dqn(net, buffersize=args.buffersize)
opt = torch.optim.Adam(agent.parameters(), args.lr)


# Start training
rewards = train(args, net, opt, env)

In [ ]:
import time

def run_trajectory(args, agent, env):
    """ Single trajectory evaluation of the agent with rendering.
    """
    # Seed
    torch.manual_seed(args.seed)
    random.seed(args.seed)
    np.random.seed(args.seed)
    env.seed(args.seed)

    reward = 0
    state = env.reset()
    done = False
    while not done:
        torch_state = agent._totorch(state, torch.float32).view(1, -1)
        action, value = agent.act(torch_state, args.end_epsilon)
        state, r, done, _ = env.step(action)
        reward += r
        env.render()
        time.sleep(1/30)
    return reward

"Reward :", run_trajectory(args, agent, env)

Plot episodic rewards obtained durin training(average of last 10 episodes)

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

plt.figure(figsize=(9, 5))
plt.plot(np.convolve(rewards, np.ones(5)/5))
plt.ylabel("Episodic reward")
plt.xlabel("episode")
plt.show()

Finally save the model

In [ ]:
torch.save(agent.state_dict(), "LunarLander_dqn_agent.b")